# KoBERT 사용하기(HuggingFace Library)
- KoBERT(SKT)도 있고 KorBERT(ETRI)도 있고 KoBART, KoGPT도 있는 것 같다. 어떤 걸 써야 하지?! 
- ETRI를 활용한 KorBertSum 
  - https://velog.io/@raqoon886/KorBertSum-SummaryBot 
  - https://github.com/raqoon886/KorBertSum
- KoBART를 이용한 요약
  - http://blog.ju-ing.co.kr/posts/KoBART-summarization/ 
  - https://younghwani.github.io/posts/kobart-summary-3/ 

In [1]:
import torch
torch.cuda.is_available()

True

Direct Access를 위한 data preprocess functions 정의하기

In [2]:
import re

def del_bracket(s):
  pattern = r'\([^)]*\)'  # ()
  s = re.sub(pattern=pattern, repl='', string=s)

  pattern = r'\[[^)]*\]'  # []
  s = re.sub(pattern=pattern, repl='', string=s)

  pattern = r'\<[^)]*\>'  # <>
  s = re.sub(pattern=pattern, repl='', string=s)

  pattern = r'\{[^)]*\}'  # {}
  s = re.sub(pattern=pattern, repl='', string=s)

  return s

def del_special_num(s):
  pattern = r'[^a-zA-Z가-힣]'
  s = re.sub(pattern=pattern, repl=' ', string=s)

  return s

def del_unit(s):
  units = ['mm', 'cm', 'km', 'ml', 'kg', 'g']
  for unit in units:
    s = s.lower() # 대문자를 소문자로 변환
    s = s.replace(unit, '')
  return s

def del_whitespace(s):
  return " ".join(s.split())

import io 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
  
def del_stopwords(s):
  stopwords = open("data/stopwords.txt", 'r', encoding="utf-8").read().split()
  #print(stopwords)
  s_o=s.split()
  s_f=[]
  for w in s_o:
    if w.strip() not in stopwords:
      s_f.append(w.strip())
  return " ".join(s_f)

In [3]:
#modelname = "beomi/KcELECTRA-base"
modelname = "klue/bert-base"
#modelname = "beomi/kcbert-base"
#modelname = "monologg/kobert"
#modelname = "kakaobrain/kogpt"

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from transformers import AdamW, get_cosine_schedule_with_warmup, BertTokenizer, BertModel

#GPU 사용
device = torch.device("cuda:0")

#BERT 모델 불러오기
bertmodel = BertModel.from_pretrained(modelname)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
import pandas as pd
#filter_values = ['기쁨', '슬픔', '분노', '불안']
#filter_values = ['기쁨', '불안', '슬픔', '분노', '상처']

train=pd.read_excel('data/Training.xlsx')
train=pd.DataFrame({'s1': train['감정_대분류'], 's2': train['감정_소분류'], 't1': train['사람문장1']})
#train = train[train['s1'].isin(filter_values)]
train.head()

val=pd.read_excel('data/Validation.xlsx')
val=pd.DataFrame({'s1': val['감정_대분류'], 's2': val['감정_소분류'], 't1': val['사람문장1']})
#val = val[val['s1'].isin(filter_values)]

Using other source:

In [6]:
data_1_path = "data/single.xlsx"
data_2_path = "data/continuous.xlsx"
# filter_values = ['공포', '분노', '슬픔', '행복', '혐오']
filter_values = ['공포', '놀람', '분노', '슬픔', '행복', '혐오', '중립']

data_1 = pd.read_excel(data_1_path)
data_1 = pd.DataFrame({'t1': data_1['Sentence'], 's1': data_1['Emotion']})
data_1 = data_1[data_1['s1'].isin(filter_values)] # filter out neutral/other typos

data_2 = pd.read_excel(data_2_path, usecols="B:C", skiprows=2, header=None, names=["t1", "s1"])
data_2 = data_2[data_2['s1'].isin(filter_values)] # filter out neutral/other typos

# print(data_2[:4])

In [7]:
#기쁨 불안 당황 슬픔 분노 상처 
s12label={'기쁨':1, '불안': 2, '당황': 3, '슬픔': 4, '분노': 5, '상처': 6}
# s12label={'기쁨':0, '불안': 1, '슬픔': 1, '분노': 1, '상처':1}
train_l=[]

for t, s in zip(train['t1'], train['s1']):
  t=del_bracket(t)
  t=del_special_num(t)
  t=del_whitespace(t)
  t=del_stopwords(t)
  train_l.append([t, s12label[s.strip()]])

for t, s in zip(val['t1'], val['s1']):
  t=del_bracket(t)
  t=del_special_num(t)
  t=del_whitespace(t)
  t=del_stopwords(t)
  train_l.append([t, s12label[s]])

# for additional data: 공포 놀람 분노 슬픔 행복 혐오 중립
s12label = {'중립': 0, '행복': 1, '공포' : 2, '슬픔': 4, '분노': 5, '놀람': 7, '혐오': 8}
# s12label = {'행복': 0, '공포': 1, '분노': 1, '슬픔': 1, '혐오': 1}
for t, s in zip(data_1['t1'], data_1['s1']):
  t=del_bracket(t)
  t=del_special_num(t)
  t=del_whitespace(t)
  t=del_stopwords(t)
  train_l.append([t, s12label[s]])

for t, s in zip(data_2['t1'], data_2['s1']):
  t=del_bracket(t)
  t=del_special_num(t)
  t=del_whitespace(t)
  t=del_stopwords(t)
  train_l.append([t, s12label[s]])

print(train_l[:10])

[['일은 끝이 없을까 화가 난다', 5], ['달에 급여가 깎였어 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가', 5], ['회사에 신입이 들어왔는데 말투가 거슬려 애를 매일 봐야 한다고 생각하니까 스트레스 받아', 5], ['직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜 일도 데 정말 분하고 섭섭해', 5], ['전 입사한 신입사원이 나를 무시하는 같아서 너무 화가', 5], ['직장에 다니고 있지만 시간만 버리는 거 같아 진지하게 진로에 대한 고민이 생겨', 5], ['성인인데도 진로를 아직도 못 정했다고 부모님이 노여워하셔 나도 섭섭해', 5], ['퇴사한 지 됐지만 천천히 직장을 구해보려고', 1], ['졸업반이라서 취업을 생각해야 하는데 지금 너무 느긋해서 이래도 되나 싶어', 2], ['요즘 직장생활이 너무 편하고 좋은 같아', 1]]


In [8]:
data_augment_flag = [1, 5, 5, 9, 5, 4, 9, 9, 9]
data_augment_limit = [150000, 150000, 150000, 150000, 150000, 150000, 150000, 150000, 150000]
# Separate the data by class
data_flag_0 = [sample for sample in train_l if sample[1] == 0]
data_flag_1 = [sample for sample in train_l if sample[1] == 1]
data_flag_2 = [sample for sample in train_l if sample[1] == 2]
data_flag_3 = [sample for sample in train_l if sample[1] == 3]
data_flag_4 = [sample for sample in train_l if sample[1] == 4]
data_flag_5 = [sample for sample in train_l if sample[1] == 5]
data_flag_6 = [sample for sample in train_l if sample[1] == 6]
data_flag_7 = [sample for sample in train_l if sample[1] == 7]
data_flag_8 = [sample for sample in train_l if sample[1] == 8]
data_flag = [data_flag_0, data_flag_1, data_flag_2, data_flag_3, data_flag_4, data_flag_5, data_flag_6, data_flag_7, data_flag_8]
print(len(data_flag_0),len(data_flag_1)/len(data_flag_0),len(data_flag_2)/len(data_flag_0),len(data_flag_3)/len(data_flag_0),len(data_flag_4)/len(data_flag_0),len(data_flag_5)/len(data_flag_0),len(data_flag_6)/len(data_flag_0),len(data_flag_7)/len(data_flag_0),len(data_flag_8)/len(data_flag_0))
augmented_data = []

48618 0.29631000863877577 0.3290756509934592 0.20165370850302358 0.35721337776132295 0.4054259739191246 0.2087704142498663 0.22139948167345427 0.11619153399975318


In [9]:
# For unbalanced-class
import random
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer

# Initialize and train WordExtractor
word_extractor = WordExtractor(min_frequency=100,
                               min_cohesion_forward=0.05, 
                               min_right_branching_entropy=0.0)
corpus = [sample[0] for sample in train_l]
word_extractor.train(corpus)
words = word_extractor.extract()

# Extract nouns from the words
nouns = list(words.keys())

# Now we'll define our text augmentation function using these nouns
def augment_korean_text(text, nouns, num_aug_samples=1):
    augmented_samples = [text]  # Start with the original text
    tokenizer = LTokenizer()

    for _ in range(num_aug_samples):
        # Replace some nouns with other random nouns
        words = tokenizer.tokenize(text)
        augmented_words = [
            word if word not in nouns else random.choice(nouns)
            for word in words
        ]
        augmented_samples.append(" ".join(augmented_words))

    return augmented_samples

training was done. used memory 1.387 Gbmory 1.146 Gb
all cohesion probabilities was computed. # words = 3508
all branching entropies was computed # words = 66604
all accessor variety was computed # words = 66604


In [10]:
i = 0
for datasets in data_flag:
    count_sample = 0
    for sample in datasets:
        text = sample[0]
        label = sample[1]
        augmented_texts = augment_korean_text(text, nouns, num_aug_samples=data_augment_flag[i])
        for augmented_text in augmented_texts:
            if count_sample < data_augment_limit[i]:
                augmented_data.append([augmented_text, label])
                count_sample += 1
    i += 1

# Shuffle the data
random.shuffle(augmented_data)

# Convert the data to a DataFrame and save it to a new Excel file
df_augmented = pd.DataFrame(augmented_data, columns=["document", "label"])
df_augmented.to_excel("data/train_l_augmentedplus_9sent.xlsx", index=False)

In [6]:
# Reload the data from the Excel file and convert it back to a list
df_augmented = pd.read_excel("data/train_l_augmented_9sent.xlsx")
train_l = df_augmented.values.tolist()

In [7]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, tokenizer, max_len):
        self.sentences = [tokenizer.encode_plus(i[sent_idx], add_special_tokens=True, max_length=max_len, padding='max_length', truncation=True) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, idx):
        token_ids = torch.tensor(self.sentences[idx]['input_ids'], dtype=torch.long)
        valid_length = torch.tensor(len(self.sentences[idx]['input_ids']), dtype=torch.long)
        segment_ids = torch.tensor(self.sentences[idx]['token_type_ids'], dtype=torch.long)
        attention_mask = torch.tensor(self.sentences[idx]['attention_mask'], dtype=torch.long)
        label = torch.tensor(self.labels[idx])
        return token_ids, valid_length, segment_ids, attention_mask, label

    def __len__(self):
        return (len(self.labels))

In [8]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 50
max_grad_norm = 1
log_interval = 10
learning_rate =  1e-5

In [9]:
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split

# Preparing the dataset
dataset_train, dataset_test = train_test_split(train_l, test_size=0.2, random_state=0)
print(len(dataset_train), len(dataset_test))
len_data = len(dataset_train) + len(dataset_test)

# Tokenization
tokenizer = AutoTokenizer.from_pretrained(modelname)

# Ensure all data is a string
dataset_train = [[str(i[0]), i[1]] for i in dataset_train]
dataset_test = [[str(i[0]), i[1]] for i in dataset_test]

# Creating instances of the BERTDataset class for train and test sets
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, max_len)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, max_len)


317228 79307


In [10]:
data_train[0]

(tensor([    2,   850,  2052,  1408,  2315, 19655,  1201,  2660,  2116,   721,
         22757,  1513,  2075,  2182,     3,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0]),
 tensor(64),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 tensor(2, dtype=torch.int32))

In [11]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=0)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=0)

In [12]:
# Set num_of_classes
num_classes = 9

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=num_classes,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids, attention_mask):
        outputs = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        pooler = outputs[1]
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [13]:
from collections import Counter
labels = []
for _, (_, _, _, _, label) in enumerate(tqdm(test_dataloader)):
    labels.extend(label.numpy())
print(Counter(labels))

100%|██████████| 1240/1240 [00:02<00:00, 423.14it/s]

Counter({5: 11795, 4: 10293, 0: 9683, 2: 9658, 7: 8749, 1: 8592, 6: 8169, 3: 7897, 8: 4471})


In [14]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.7).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(out, label):
    predicted = out.argmax(1)
    correct = (predicted == label).sum().item()
    total = label.size(0)
    accuracy = correct / total
    # print(f'Predicted: {predicted}, Label: {label}, Correct: {correct}, Total: {total}, Accuracy: {accuracy}')
    return accuracy

    
train_dataloader

c:\Users\Chiwoong\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [15]:
# For time-based tensorboard store
num_augmentations="134"
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d")
log_dir = f'./logs/{timestamp}-model{modelname}-data{len_data}/a{num_augmentations}-class{num_classes}-batch{batch_size}-lr{learning_rate}-epoch{num_epochs}-maxlen{max_len}'

In [16]:
from torch.utils.tensorboard import SummaryWriter

In [17]:
from tqdm.notebook import tqdm

best_acc = 0
max_patience = 5
patience = 0
PATH = log_dir
writer = SummaryWriter(PATH)

# print("Starting training loop...")
for e in range(num_epochs):
    # print("Starting epoch ", e+1)
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    # for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
    for batch_id, (token_ids, valid_length, segment_ids, attention_mask, label) in enumerate(tqdm(train_dataloader)):
        # print("Starting batch ", batch_id+1)    
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        attention_mask = attention_mask.to(device)
        out = model(token_ids, valid_length, segment_ids, attention_mask)
        # print(f'Output: {out.argmax(1)}, Label: {label}')

        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            writer.add_scalar('loss/train_loss', loss.data.cpu().numpy(), e*len(train_dataloader)+batch_id+1)
            writer.add_scalar('acc/train_acc', train_acc / (batch_id+1), e*len(train_dataloader)+batch_id+1)
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, attention_mask, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        attention_mask = attention_mask.to(device)
        out = model(token_ids, valid_length, segment_ids, attention_mask)
        # print(f'Output: {out.argmax(1)}, Label: {label}')
        
        test_acc += calc_accuracy(out, label)
        loss = loss_fn(out, label)
    writer.add_scalar('acc/test_acc', test_acc / (batch_id+1), e+1)        
    writer.add_scalar('loss/test_loss', loss.data.cpu().numpy(), e+1)        
    print("epoch {} loss {} test acc {}".format(e+1, loss.data.cpu().numpy(), test_acc / (batch_id+1)))
    if test_acc/(batch_id+1)>best_acc:
      best_acc=test_acc/(batch_id+1)
      patience=0
      torch.save(model, PATH + '{}_{}_model.pt'.format(e+1, test_acc/(batch_id+1)))
    else:
      patience+=1
    if patience>max_patience:
      break
    test_acc = 0  # Reset test accuracy for the next epoch

  0%|          | 0/4957 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.3819549083709717 train acc 0.078125
epoch 1 batch id 11 loss 2.4706594944000244 train acc 0.11789772727272728
epoch 1 batch id 21 loss 2.3957042694091797 train acc 0.12872023809523808
epoch 1 batch id 31 loss 2.481553554534912 train acc 0.125
epoch 1 batch id 41 loss 2.2396457195281982 train acc 0.12233231707317073
epoch 1 batch id 51 loss 2.3417234420776367 train acc 0.12193627450980392
epoch 1 batch id 61 loss 2.209585666656494 train acc 0.12320696721311475
epoch 1 batch id 71 loss 2.2701315879821777 train acc 0.11971830985915492
epoch 1 batch id 81 loss 2.363008499145508 train acc 0.12017746913580248
epoch 1 batch id 91 loss 2.394219160079956 train acc 0.11916208791208792
epoch 1 batch id 101 loss 2.445164442062378 train acc 0.11834777227722772
epoch 1 batch id 111 loss 2.274477958679199 train acc 0.11866554054054054
epoch 1 batch id 121 loss 2.3957669734954834 train acc 0.11854338842975207
epoch 1 batch id 131 loss 2.3108887672424316 train acc 0.1182013358

  0%|          | 0/1240 [00:00<?, ?it/s]

epoch 1 loss 1.7997475862503052 test acc 0.4208749541788856


  0%|          | 0/4957 [00:00<?, ?it/s]

...

epoch 49 loss 0.5431835651397705 test acc 0.8529382789589444


  0%|          | 0/4957 [00:00<?, ?it/s]

epoch 50 batch id 1 loss 0.044347457587718964 train acc 0.984375
epoch 50 batch id 11 loss 0.01645469479262829 train acc 0.9857954545454546
epoch 50 batch id 21 loss 0.062406543642282486 train acc 0.9858630952380952
epoch 50 batch id 31 loss 0.04633522406220436 train acc 0.9858870967741935
epoch 50 batch id 41 loss 0.038612086325883865 train acc 0.9870426829268293
epoch 50 batch id 51 loss 0.011371780186891556 train acc 0.9868259803921569
epoch 50 batch id 61 loss 0.07804035395383835 train acc 0.9841188524590164
epoch 50 batch id 71 loss 0.0694441944360733 train acc 0.9848151408450704
epoch 50 batch id 81 loss 0.026192741468548775 train acc 0.9839891975308642
epoch 50 batch id 91 loss 0.06548288464546204 train acc 0.984375
epoch 50 batch id 101 loss 0.03494243696331978 train acc 0.9839108910891089
epoch 50 batch id 111 loss 0.0005955218803137541 train acc 0.9836711711711712
epoch 50 batch id 121 loss 0.13762372732162476 train acc 0.9837293388429752
epoch 50 batch id 131 loss 0.01901318

  0%|          | 0/1240 [00:00<?, ?it/s]

epoch 50 loss 0.5398192405700684 test acc 0.8529382789589444


To load the model from PATH?

# Interactive Test

In [6]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from transformers import AdamW, get_cosine_schedule_with_warmup, BertTokenizer, BertModel
from collections import Counter
from torch.nn.utils.rnn import pad_sequence
from transformers import AutoTokenizer
import re
import io 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

In [7]:
def del_bracket(s):
  pattern = r'\([^)]*\)'  # ()
  s = re.sub(pattern=pattern, repl='', string=s)

  pattern = r'\[[^)]*\]'  # []
  s = re.sub(pattern=pattern, repl='', string=s)

  pattern = r'\<[^)]*\>'  # <>
  s = re.sub(pattern=pattern, repl='', string=s)

  pattern = r'\{[^)]*\}'  # {}
  s = re.sub(pattern=pattern, repl='', string=s)

  return s

def del_special_num(s):
  pattern = r'[^a-zA-Z가-힣]'
  s = re.sub(pattern=pattern, repl=' ', string=s)

  return s

def del_unit(s):
  units = ['mm', 'cm', 'km', 'ml', 'kg', 'g']
  for unit in units:
    s = s.lower() # 대문자를 소문자로 변환
    s = s.replace(unit, '')
  return s

def del_whitespace(s):
  return " ".join(s.split())
  
def del_stopwords(s):
  stopwords = open("data/stopwords.txt", 'r', encoding="utf-8").read().split()
  #print(stopwords)
  s_o=s.split()
  s_f=[]
  for w in s_o:
    if w.strip() not in stopwords:
      s_f.append(w.strip())
  return " ".join(s_f)

In [9]:
modelname = "klue/bert-base"
max_length = 64
num_classes = 6

#GPU 사용
device = torch.device("cuda:0")

#BERT 모델 불러오기
bertmodel = BertModel.from_pretrained(modelname)

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=num_classes,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids, attention_mask):
        outputs = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        pooler = outputs[1]
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
# assuming you have a function for tokenization
tokenizer = AutoTokenizer.from_pretrained(modelname)

# Define a function to get the valid_length, attention_mask and segment_ids
def get_inputs(tokens):
    tokens = ['[CLS]'] + tokens + ['[SEP]']
    valid_length = len(tokens)
    segment_ids = [0]*valid_length
    attention_mask = [1]*valid_length

    # Pad up to max length
    if valid_length < max_length:
        pad_length = max_length - valid_length
        tokens.extend(['[PAD]' for _ in range(pad_length)])
        attention_mask.extend([0]*pad_length)
        segment_ids.extend([0]*pad_length)

    # Convert tokens to IDs
    token_ids = tokenizer.convert_tokens_to_ids(tokens)

    return torch.tensor([token_ids], dtype=torch.long), torch.tensor([valid_length], dtype=torch.long), torch.tensor([segment_ids], dtype=torch.long), torch.tensor([attention_mask], dtype=torch.long)

# assuming you have a function for pre-processing
def preprocess(text):
    for t in text:
        t=del_bracket(t)
        t=del_special_num(t)
        t=del_whitespace(t)
        t=del_stopwords(t)
    return text.lower()

# Load text
with open('data/test1.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Preprocess and tokenize the text
tokens = tokenizer.tokenize(preprocess(text))
if len(tokens) > max_length-2: # Account for [CLS] and [SEP]
    tokens = tokens[:max_length-2]

# Get inputs
token_ids, valid_length, segment_ids, attention_mask = get_inputs(tokens)

# Load the model
model_path = 'logs/20230519-modelklue/bert-base-data291355/a4-class6-batch64-lr1e-05-epoch50-maxlen6447_0.9486130537162282_model.pt'  # replace with your actual path
model = torch.load(model_path)

# Switch to eval mode
model.eval()

# Determine if a cuda-capable GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Load the model and move to the GPU if available
model = torch.load(model_path)
model.to(device)

# Move all your tensors to the same device as your model
token_ids = token_ids.to(device)
valid_length = valid_length.to(device)
segment_ids = segment_ids.to(device)
attention_mask = attention_mask.to(device)

# Ensure no gradient is calculated
with torch.no_grad():
    sentiment_vector = model(token_ids, valid_length, segment_ids, attention_mask)

print(sentiment_vector)


cuda
tensor([[-1.1051, -2.7562,  9.9050, -2.4808, -3.0826,  0.3052]],
       device='cuda:0')


In [11]:
import torch.nn.functional as F

# Get the output from your model
output = sentiment_vector

# Apply softmax to convert output to probabilities
probabilities = F.softmax(output, dim=-1)

emotions = ['기쁨', '불안', '슬픔', '분노', '상처', '당황'] 

# Get the indices that would sort the probability tensor
sorted_indices = torch.argsort(probabilities, dim=-1, descending=True)

primary_emotion_idx = sorted_indices[0][0].item()
secondary_emotion_idx = sorted_indices[0][1].item()

primary_emotion = emotions[primary_emotion_idx]
primary_emotion_probability = probabilities[0][primary_emotion_idx].item()

secondary_emotion = emotions[secondary_emotion_idx]
secondary_emotion_probability = probabilities[0][secondary_emotion_idx].item()

print(f"당신은 지금 {primary_emotion}을 느끼고 있네요.")
print(f"주 감정: {primary_emotion_probability * 100:.2f}% {primary_emotion}, 부 감정: {secondary_emotion_probability * 100:.2f}% {secondary_emotion}")


당신은 지금 슬픔을 느끼고 있네요.
주 감정: 99.99% 슬픔, 부 감정: 0.01% 당황


## Make music library with lyrics

In [12]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Load the music data
df_music = pd.read_csv("lyrics_indi(1-22000).csv")

def process_lyrics(lyrics, tokenizer, model):
    # Preprocess and tokenize the lyrics
    tokens = tokenizer.tokenize(preprocess(lyrics))
    if len(tokens) > max_length-2: # Account for [CLS] and [SEP]
        tokens = tokens[:max_length-2]

    # Get inputs
    token_ids, valid_length, segment_ids, attention_mask = get_inputs(tokens)

    # Move all your tensors to the same device as your model
    token_ids = token_ids.to(device)
    valid_length = valid_length.to(device)
    segment_ids = segment_ids.to(device)
    attention_mask = attention_mask.to(device)

    # Ensure no gradient is calculated
    with torch.no_grad():
        sentiment_vector = model(token_ids, valid_length, segment_ids, attention_mask)

    return sentiment_vector.detach().cpu().numpy()[0]  # Return as a 1-D numpy array

# Process all songs and add a new column for the sentiment vectors
df_music['sentiment_vector'] = df_music['lyrics'].apply(lambda x: process_lyrics(x, tokenizer, model))

# Save to a new CSV file
df_music.to_csv("music_library.csv", index=False)



Token indices sequence length is longer than the specified maximum sequence length for this model (614 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
# Function to compute cosine similarities and retrieve the top 10 songs
def recommend_songs(diary_text):
    # Process the diary text
    diary_vector = process_text(diary_text, tokenizer, model)

    # Compute cosine similarities
    similarities = cosine_similarity([diary_vector], df_music['sentiment_vector'].to_list())

    # Get the top 10 song indices
    top_10_indices = similarities[0].argsort()[-10:][::-1]

    # Return the corresponding songs
    return df_music.iloc[top_10_indices]

# Function to print out the recommended songs
def print_recommended_songs(diary_text):
    recommended_songs = recommend_songs(diary_text)
    for i, song in recommended_songs.iterrows():
        similarity = cosine_similarity([process_text(diary_text, tokenizer, model)], [song['sentiment_vector']])[0][0]
        print(f"Title: {song['title']}, Artist: {song['artist']}, Lyrics: {song['lyrics']}, Similarity: {similarity*100:.2f}%")

# Test with a diary text
with open('data/test1.txt', 'r', encoding='utf-8') as f:
    diary_text = f.read()

print_recommended_songs(diary_text)